In [36]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
import mylib

In [37]:
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)

In [38]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19661 entries, 0 to 19660
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             19661 non-null  object
 1   pub_title      19661 non-null  object
 2   dataset_title  19661 non-null  object
 3   dataset_label  19661 non-null  object
 4   cleaned_label  19661 non-null  object
dtypes: object(5)
memory usage: 768.1+ KB


In [39]:
gt_map: Dict[str, List[str]] = {}
sep = "|"
for t in tqdm(train.itertuples()):
    pid = getattr(t, "Id")
    cl = getattr(t, "cleaned_label")
    if pid not in gt_map:
        gt_map[pid] = list()
    gt_map[pid].append(mylib.clean_text(cl))

19661it [00:00, 211609.47it/s]


In [40]:
tmp = []
for k, v in tqdm(gt_map.items()):
    is_multi = 0
    if len(v) > 1:
        is_multi = 1
    vs = "|".join(sorted(v))
    tmp.append((k, vs, is_multi))

100%|██████████| 14316/14316 [00:00<00:00, 588019.94it/s]


In [41]:
df = pd.DataFrame(tmp, columns=["pid", "ground_truth", "is_multi"])
df["is_multi"] = df["is_multi"].astype(np.int8) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14316 entries, 0 to 14315
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pid           14316 non-null  object
 1   ground_truth  14316 non-null  object
 2   is_multi      14316 non-null  int8  
dtypes: int8(1), object(2)
memory usage: 237.8+ KB


In [42]:
df.sample(20).head(20)

,pid,ground_truth,is_multi
13165,f01dc252-8d0d-4421-b2a5-435f5fb11ec7,census of agriculture,0
3642,e9a2d09b-6d00-4331-85bd-256a8ef1c29d,adni|alzheimer s disease neuroimaging initiative adni,1
3665,f191a583-f30b-4fe9-a2bf-06b473cfe3ca,adni|alzheimer s disease neuroimaging initiative adni,1
5362,45e9649a-cb87-499d-a657-8d09057ad386,adni,0
3273,81a1e992-e4b3-49ae-91a8-ae233aa44d26,adni,0
324,649d96d9-15d1-4f5b-b76d-1878c354514e,slosh model,0
12739,6a4fad01-fece-4b7e-84bf-83a1efe519f7,census of agriculture,0
6164,b0b77efe-b981-4f43-b0a5-7432ef0f2980,adni|alzheimer s disease neuroimaging initiative adni,1
12131,d61f1360-8217-4d9d-8760-11122011f0d0,noaa optimum interpolation sea surface temperature|optimum interpolation sea surface temperature,1
2230,3b30f42a-0cd7-4f76-9c5f-445cff1df70f,baltimore longitudinal study of aging,0


In [43]:
df["is_multi"].value_counts(normalize=True)

0    0.667994
1    0.332006
Name: is_multi, dtype: float64

In [44]:
df.to_parquet("output/train.parquet", index=False)